In [1]:
import numpy as np
import os,cv2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import visualkeras

In [2]:
from keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import plot_model, load_img, to_categorical

In [ ]:
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
data_dir="Face-CK/"
labels=[]
for i in os.listdir(data_dir):
    labels.append(i)
no_labels=len(labels)
print("No of labels : ",no_labels)
print("Labels : ",labels)

In [18]:
images=[]
for i in os.listdir(data_dir):
    imgs=os.listdir(data_dir+"/"+i)
    for img in imgs:
        input_img=cv2.imread(data_dir+"/"+i+"/"+img)
        input_img=cv2.resize(input_img,(48,48))
        images.append(input_img)

img_array=np.array(images)
# img_array=img_array.astype('float32')
img_array=img_array/255.
img_array.shape
# img_array

(981, 48, 48, 3)

In [16]:
img=img_array.shape[0]
faces=np.ones((img,),dtype='float32')

faces[0:134]=0 #135(angry)
faces[135:188]=1 #54(contempt)
faces[189:365]=2 #177(disgust)
faces[366:440]=3 #75(fear)
faces[441:647]=4 #207(happy)
faces[648:731]=5 #84(sadness)
faces[732:980]=6 #249(surprise)

In [ ]:
y=to_categorical(faces,7)
x,y=shuffle(img_array,y,random_state=1)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15,random_state=1)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(x_train)

In [ ]:
augmented_images = datagen.flow(x_train, y_train, batch_size=7)

# Example: Display a few augmented images
import matplotlib.pyplot as plt
fig, ax = plt.subplots(2, 3, figsize=(20, 7))
for i in range(2):
    for j in range(3):
        augmented_sample = augmented_images.next()
        ax[i, j].imshow(augmented_sample[0][0])
        ax[i, j].axis('off')
plt.show()

In [ ]:
modal=Sequential()
modal.add(Conv2D(32, kernel_size=(3,3), activation='relu',input_shape=(48,48, 3)))
modal.add(MaxPool2D(2,2))
modal.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
modal.add(MaxPool2D(2,2))

modal.add(Flatten())
modal.add(Dense(128,activation='relu'))
modal.add(Dense(7,activation='softmax'))

In [ ]:
modal.summary()

In [ ]:
visualkeras.layered_view(modal)

In [ ]:
modal.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
class myCallback(Callback):
    def on_epoch_end(self,epoch,logs={}):
        if logs.get('accuracy')>=0.90:
            print("\nReached More than 90% accuraccy")
            self.model.stop_training=True
callback=myCallback()

In [ ]:
hist=modal.fit(
    datagen.flow(x_train,y_train,batch_size=7),
    epochs=10000,
    validation_data=(x_test, y_test),
    callbacks=callback
)

In [ ]:
plt.plot(hist.history['accuracy'],color='red')
plt.plot(hist.history['val_accuracy'],color='orange',linestyle="dashed")
plt.plot(hist.history['loss'],color='green')
plt.plot(hist.history['val_loss'],color='blue',linestyle="dashed")

In [ ]:
y_pred=modal.predict(x_test)
y_pred=[np.argmax(probas) for probas in y_pred]
y_test=[np.argmax(probas) for probas in y_test]

In [ ]:
cmp=confusion_matrix(y_test,y_pred)
cmp

In [ ]:
plt.figure()
import seaborn as sns
sns.heatmap(cmp,xticklabels=labels,yticklabels=labels,annot=True,fmt='d')
plt.title("Confusion Matrix",color="darkred")
plt.show()

In [ ]:
import random
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import load_img
happy_train=os.path.join(data_dir,'disgust')
file=os.listdir(happy_train)
sample=random.choice(file)
img=load_img(os.path.join(happy_train,sample),target_size=(48,48))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
val=modal.predict(x)
# ['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']
print(val)
plt.imshow(img)
if val[0][0] == 1:
    print("Anger")
elif val[0][1]==1:
    print("Contempt")
elif val[0][2]==1:
    print("disgust")
elif val[0][3]==1:
    print("fear")
elif val[0][4]==1:
    print("happy")
elif val[0][5]==1:
    print("sadness")
elif val[0][6]==1:
    print("surprise")
else:
    print("Couldn't identify")

In [ ]:
modal.save("face_emotion.h5")

In [ ]:
import cv2
import numpy as np
from keras.models import load_model

# Load the pre-trained emotion detection model
model = load_model('./face_emotion.h5')

# Load the haarcascades for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

image_path = "./Images/sreekar.jpg"
image = cv2.imread(image_path)
if image is not None:
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    num_labels = 7

    for (x, y, w, h) in faces:
        roi = image[y:y+h, x:x+w]

        framing = cv2.resize(roi, (48, 48))
        framing = np.expand_dims(framing, axis=0)

        # Predict emotion
        emotion_probabilities = model.predict(framing)
        predicted_label_index = np.argmax(emotion_probabilities)
        predicted_label = labels[predicted_label_index]

        # Display emotion on the image
        cv2.putText(image, f'Emotion: {predicted_label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Display the image with emotions
    cv2.imshow('Facial Emotion Recognition', image)
    plt.imshow(image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Error to load the image/update the permissions")

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
model = load_model('./face_emotion.h5')

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
num_labels = 7

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi = frame[y:y+h, x:x+w]
        framing = cv2.resize(roi, (48, 48))
        framing = np.expand_dims(framing, axis=0)

        # Predict emotion
        emotion_probabilities = model.predict(framing)
        predicted_label_index = np.argmax(emotion_probabilities)
        predicted_label = labels[predicted_label_index]

        # Display emotion on the frame
        cv2.putText(frame, f'Emotion: {predicted_label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Facial Emotion Recognition', frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
